In [2]:
pip install cobra


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 8.6 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 330.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 496.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.0 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 1.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 4.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00

Import of libraries needed for importing the model and defining the model

In [6]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model = read_sbml_model('iJO1366.xml')

In [3]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1


In [4]:
model.optimize() #initial optimal solution is 0.982

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,0.000000e+00
RNTR2c2,0.026541,2.775558e-17


Defining the metabolites

In [5]:
ethylene = Metabolite(id = 'eth_c', compartment='c', name='ethylene', formula='C2H4')
hydroxylarginine = Metabolite(id = 'hlarg_c', compartment='c', name='5-hydroxy-l-arginine', formula='C6H14N4O3')
guanidine = Metabolite(id = 'guan_c', compartment='c', name='guanidine', formula='C1H5N3')
ldelta1pyrroline5carboxylate = Metabolite(id = 'ld1p5carb_c', compartment='c', name='l-delta-1-pyrroline-5-carboxylate', formula='C5H7N1O2')

Defining the reactions

In [6]:
Ethylene_forming_enzyme_main = Reaction('EFE_m')
Ethylene_forming_enzyme_side = Reaction('EFE_s')
Non_enzymatic_hlarg_degradation = Reaction('neHLARGdeg')

Adding the metabolites to the reactions

In [7]:
Ethylene_forming_enzyme_main.add_metabolites({model.metabolites.akg_c: -2,              
                              model.metabolites.o2_c: -2,
                              ethylene: 2,
                              model.metabolites.co2_c: 6,
                              model.metabolites.o2_c: 1,
                             })

Ethylene_forming_enzyme_side.add_metabolites({model.metabolites.akg_c: -1,              
                              model.metabolites.o2_c: -1,
                              model.metabolites.arg__L_c: -1,
                              hydroxylarginine: 1,
                              model.metabolites.co2_c: 1,
                              model.metabolites.succ_c: 1,
                             })

Non_enzymatic_hlarg_degradation.add_metabolites({hydroxylarginine: -1,
                              ldelta1pyrroline5carboxylate: 1,
                              guanidine: 1,
                              model.metabolites.h2o_c: 1,
                             })

Adding the reactions to the model

In [8]:
model.add_reactions([Ethylene_forming_enzyme_main])
model.add_reactions([Ethylene_forming_enzyme_side])
model.add_reactions([Non_enzymatic_hlarg_degradation])

In [9]:
model.add_boundary(model.metabolites.eth_c, type='demand')
model.add_boundary(model.metabolites.hlarg_c, type='demand')
model.add_boundary(model.metabolites.guan_c, type='demand')
model.add_boundary(model.metabolites.ld1p5carb_c, type='demand')

Reaction identifier,DM_ld1p5carb_c
Name,l-delta-1-pyrroline-5-carboxylate demand
Memory address,0x7f5074827b80
Stoichiometry,ld1p5carb_c --> l-delta-1-pyrroline-5-carboxylate -->
GPR,
Lower bound,0
Upper bound,1000.0


In [10]:
for metabolite in model.metabolites:
    print(metabolite.id, metabolite.name) #this is to check whether the metabolites have been added to the model

10fthf_c 10-Formyltetrahydrofolate
12dgr120_c 1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)
12dgr140_c 1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)
12dgr141_c 1,2-Diacyl-sn-glycerol (ditetradec-7-enoyl, n-C14:1)
12dgr160_c 1,2-Diacyl-sn-glycerol (dihexadecanoyl, n-C16:0)
12dgr161_c 1,2-Diacyl-sn-glycerol (dihexadec-9-enoyl, n-C16:1)
12dgr180_c 1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)
12dgr181_c 1,2-Diacyl-sn-glycerol (dioctadec-11-enoyl, n-C18:1)
12ppd__R_c (R)-Propane-1,2-diol
12ppd__S_c (S)-Propane-1,2-diol
13dpg_c 3-Phospho-D-glyceroyl phosphate
14dhncoa_c 1,4-dihydroxy-2-napthoyl-CoA
14glucan_c 1,4-alpha-D-glucan
15dap_c 1,5-Diaminopentane
1ddecg3p_c 1-dodecanoyl-sn-glycerol 3-phosphate
1hdec9eg3p_c 1-hexadec-9-enoyl-sn-glycerol 3-phosphate
1hdecg3p_c 1-hexadecanoyl-sn-glycerol 3-phosphate
1odec11eg3p_c 1-octadec-11-enoyl-sn-glycerol 3-phosphate
1odecg3p_c 1-octadecanoyl-sn-glycerol 3-phosphate
1pyr5c_c 1-Pyrroline-5-carboxylate
5dh4dglc_c 5-Dehydro-4-deoxy-D-glucarat

In [11]:
model.reactions.EFE_m #to check if main EFE reaction is added -> yes it is, but O2 is missing on educt side


Reaction identifier,EFE_m
Name,
Memory address,0x7f5074825a80
Stoichiometry,2 akg_c --> 6 co2_c + 2 eth_c + o2_c 2 2-Oxoglutarate --> 6 CO2 CO2 + 2 ethylene + O2 O2
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
model.reactions.EFE_s #to check if main EFE reaction is added -> yes it is, but O2 is missing on educt side

Reaction identifier,EFE_s
Name,
Memory address,0x7f50747db400
Stoichiometry,akg_c + arg__L_c + o2_c --> co2_c + hlarg_c + succ_c 2-Oxoglutarate + L-Arginine + O2 O2 --> CO2 CO2 + 5-hydroxy-l-arginine + Succinate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
model.reactions.neHLARGdeg

Reaction identifier,neHLARGdeg
Name,
Memory address,0x7f5074825240
Stoichiometry,hlarg_c --> guan_c + h2o_c + ld1p5carb_c 5-hydroxy-l-arginine --> guanidine + H2O H2O + l-delta-1-pyrroline-5-carboxylate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
model.optimize() #to check if adding metabolites and reactions had an effect on biomass -> it hasnt, good

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-0.296557
EX_co2_e,19.675223,0.000000
EX_cobalt2_e,-0.000025,0.000000
DM_4crsol_c,0.000219,0.000000
...,...,...
neHLARGdeg,0.000000,0.000000
DM_eth_c,0.000000,0.000000
DM_hlarg_c,0.000000,0.000000
DM_guan_c,0.000000,0.000000


Create a  xml file with the modified model

In [4]:
from cobra.io import write_sbml_model

In [7]:
write_sbml_model(model, "modified_model.xml")